In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading shadow.p and target_model

In [2]:
import pickle
import torch

DATA_PATH = "/content/drive/MyDrive/pickle/cifar10/resnet34/shadow.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=64, shuffle=False, num_workers=2)

for batch_idx, (img, label) in enumerate(dataloader):
    img = img.to(device)

In [3]:
import torchvision.models as models


MODEL_PATH = "/content/drive/MyDrive/models/resnet34_cifar10.pth"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

target_model = models.resnet34(num_classes=10).to(device)

state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])
acc = state_dict['acc']
epoch = state_dict['epoch']



## Shadow model

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [6]:
train_data, test_data = train_test_split(dataset, test_size=0.36, random_state=42)

transform = transforms.Compose([
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CustomDataset(train_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)


In [7]:
model = models.resnet34(pretrained=True) #import resnet34


num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10) #change to match our data

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 102MB/s]


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [9]:
print(len(train_loader))

384


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 40

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print("Finished Training")


Epoch 1, Loss: 1.1940001441786687
Epoch 2, Loss: 0.6700422165837759
Epoch 3, Loss: 0.45855264257018763
Epoch 4, Loss: 0.3207520821209376
Epoch 5, Loss: 0.23335544646639997
Epoch 6, Loss: 0.16996072488836944
Epoch 7, Loss: 0.13757685246916176
Epoch 8, Loss: 0.1175926087720048
Epoch 9, Loss: 0.10179240263823885
Epoch 10, Loss: 0.08125136534605797
Epoch 11, Loss: 0.08270862133213086
Epoch 12, Loss: 0.06074082056632809
Epoch 13, Loss: 0.06570242371223382
Epoch 14, Loss: 0.057361919908241056
Epoch 15, Loss: 0.054369070595081816
Epoch 16, Loss: 0.05538081398056723
Epoch 17, Loss: 0.04442037922126474
Epoch 18, Loss: 0.04710636304783596
Epoch 19, Loss: 0.04710769882967725
Epoch 20, Loss: 0.04594998536807301
Epoch 21, Loss: 0.04023627380972054
Epoch 22, Loss: 0.038240819712427765
Epoch 23, Loss: 0.03633620104240739
Epoch 24, Loss: 0.03809963184274542
Epoch 25, Loss: 0.038942404278107766
Epoch 26, Loss: 0.02772755377751916
Epoch 27, Loss: 0.03272996057406393
Epoch 28, Loss: 0.03032018743435098
E

In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 80.71296296296296%


In [12]:
torch.save(model.state_dict(), 'most_recent_shadow.pth') #saving the model weights

## Creating dataset to train attack model

In [13]:
attack_model_dataset = []
print(len(attack_model_dataset))

0


In [14]:
import torch.nn.functional as F
attack_model_dataset = []
cnt = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        probabilities = F.softmax(outputs, dim=1)
        for i in range(len(images)):
          attack_model_dataset.append(probabilities[i].tolist()+[0]) #0 because it is test data, non-member for the shadow model





In [15]:
print(len(attack_model_dataset))

10800


In [16]:
cnt = 0
with torch.no_grad():
    cnt=0
    for images, labels in train_loader:
        cnt+=1
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        probabilities = F.softmax(outputs, dim=1)
        for i in range(len(images)):
          attack_model_dataset.append(probabilities[i].tolist()+[1]) #1 because it is train data, member for the shadow model




In [17]:
print(len(attack_model_dataset))

30000


## Attack Model

In [18]:
class CustomAttackDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor = self.data[idx][:-1]
        label = self.data[idx][-1]
        return torch.tensor(tensor), label


attack_dataset = CustomAttackDataset(attack_model_dataset)

In [19]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.sigmoid(x)
        return x

attack_model = BinaryClassifier()


In [20]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch.utils.data import random_split

train_size = int(0.95 * len(attack_dataset))
test_size = len(attack_dataset) - train_size
train_attack_dataset, test_attack_dataset = random_split(attack_dataset, [train_size, test_size])

train_attack_loader = DataLoader(train_attack_dataset, batch_size=16, shuffle=True)
test_attack_loader = DataLoader(test_attack_dataset, batch_size=16, shuffle=False)

print("train shape:", len(train_attack_dataset))

print("test shape:", len(test_attack_dataset))

train shape: 28500
test shape: 1500


In [21]:
print(train_attack_dataset[0])

(tensor([2.0825e-03, 8.1222e-06, 1.3062e-04, 1.9248e-05, 1.2406e-07, 9.9753e-01,
        1.7637e-07, 1.7143e-04, 5.0896e-05, 9.0517e-06]), 1)


In [22]:
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=0.0001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
attack_model.to(device)

num_epochs = 60

for epoch in range(num_epochs):
    attack_model.train()
    running_loss = 0.0
    for tensors, labels in train_attack_loader:
        tensors, labels = tensors.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = attack_model(tensors).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_attack_loader)}")

print("Finished Training")


Epoch 1, Loss: 0.649994349539882
Epoch 2, Loss: 0.6326193025484096
Epoch 3, Loss: 0.5964781638113604
Epoch 4, Loss: 0.5847617559695217
Epoch 5, Loss: 0.5807793198276717
Epoch 6, Loss: 0.5791314361004449
Epoch 7, Loss: 0.5784545913756763
Epoch 8, Loss: 0.5783849672469761
Epoch 9, Loss: 0.5774542195461146
Epoch 10, Loss: 0.5769557226470157
Epoch 11, Loss: 0.5761402792032854
Epoch 12, Loss: 0.5758175578148143
Epoch 13, Loss: 0.5752691899485861
Epoch 14, Loss: 0.5754783342014392
Epoch 15, Loss: 0.5740189875918206
Epoch 16, Loss: 0.5744686200127724
Epoch 17, Loss: 0.5734735580138218
Epoch 18, Loss: 0.5729668593634125
Epoch 19, Loss: 0.5729746277685519
Epoch 20, Loss: 0.5722451736830702
Epoch 21, Loss: 0.5718330847461082
Epoch 22, Loss: 0.5713764180906962
Epoch 23, Loss: 0.5714927615073125
Epoch 24, Loss: 0.570631872958741
Epoch 25, Loss: 0.5701288083018411
Epoch 26, Loss: 0.5700926613292576
Epoch 27, Loss: 0.569750752095154
Epoch 28, Loss: 0.5688261394080624
Epoch 29, Loss: 0.56912228540338

In [23]:
attack_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for tensors, labels in test_attack_loader:
        tensors, labels = tensors.to(device), labels.to(device).float()
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 72.86666666666666%


## Load the eval.p file and find accuracy

In [24]:
DATA_PATH = "/content/drive/MyDrive/pickle/cifar10/resnet34/eval.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    eval_dataset = pickle.load(f)

eval_dataloader = torch.utils.data.DataLoader(
    eval_dataset, batch_size=64, shuffle=True, num_workers=2)


In [25]:
print(len(eval_dataset))

200


In [26]:
import torch.nn.functional as F
target_model.eval()
attack_model_eval_dataset = []
total=0
correct = 0
with torch.no_grad():
    for images, labels, membership in eval_dataloader:

        images, labels = images.to(device), labels.to(device)
        outputs = target_model(images)
        #print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        probabilities = F.softmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for i in range(len(images)):
          attack_model_eval_dataset.append(probabilities[i].tolist()+[membership[i]])

print("Accuracy of target model :", 100*(correct/total))

Accuracy of target model : 85.5


In [27]:
attack_eval_dataset = CustomAttackDataset(attack_model_eval_dataset)

In [28]:
attack_eval_loader = DataLoader(attack_eval_dataset, batch_size=10, shuffle=False)

In [29]:
attack_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for tensors, labels in attack_eval_loader:
        tensors, labels = tensors.to(device), labels.to(device).float()
        outputs = attack_model(tensors).squeeze()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 68.0%


## Load test.p file and generate predictions

In [30]:
DATA_PATH = "/content/drive/MyDrive/pickle/cifar10/resnet34/test.p"

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open(DATA_PATH, "rb") as f:
    test_dataset = pickle.load(f)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=False, num_workers=2)


In [31]:
import torch.nn.functional as F
target_model.eval()
final_test_dataset = []
total=0
correct = 0
with torch.no_grad():
    cnt=0
    for images, labels in test_dataloader:

        images, labels = images.to(device), labels.to(device)
        outputs = target_model(images)
        #print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        probabilities = F.softmax(outputs, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for i in range(len(images)):
          final_test_dataset.append(probabilities[i].tolist())

In [32]:
print(final_test_dataset[0])

[5.531347414944321e-05, 0.00011613118840614334, 0.00021471142827067524, 0.0001690014760242775, 0.00030225186492316425, 0.0006084183696657419, 0.9982144832611084, 8.302015339722857e-05, 6.725761340931058e-05, 0.00016943008813541383]


In [33]:
import torch
from torch.utils.data import Dataset

class FinalTestDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor = self.data[idx]
        return torch.tensor(tensor, dtype=torch.float32)

final_test_dataset = FinalTestDataset(final_test_dataset)
final_test_loader = torch.utils.data.DataLoader(final_test_dataset, batch_size=16, shuffle=False)


In [34]:
print(final_test_dataset[0])

tensor([5.5313e-05, 1.1613e-04, 2.1471e-04, 1.6900e-04, 3.0225e-04, 6.0842e-04,
        9.9821e-01, 8.3020e-05, 6.7258e-05, 1.6943e-04])


## Save predicted values

In [35]:
import torch
import numpy as np

attack_model.eval()
predicted_values = []

with torch.no_grad():
    for prob in final_test_loader:
        prob = prob.to(device)
        outputs = attack_model(prob).squeeze()
        predicted = (outputs > 0.5).float()
        predicted_values.extend(predicted.cpu().numpy())

predicted_array = np.array(predicted_values)
print(predicted_array)
np.save('predicted_values.npy', predicted_array)


[1. 0. 0. ... 0. 1. 1.]


In [36]:
print(predicted_array[:100])

[1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 1.]
